In [8]:
from utils.plot_script import *
from utils import paramUtil
from utils.utils import *
from utils.quaternion import *


def plot_t2m(mp_data, result_path, caption):
        mp_joint = []
        for i, data in enumerate(mp_data):
            if i == 0:
                joint = data[:,:22*3].reshape(-1,22,3)
            else:
                joint = data[:,:22*3].reshape(-1,22,3)

            mp_joint.append(joint)

        plot_3d_motion(result_path, paramUtil.t2m_kinematic_chain, mp_joint, title=caption, fps=30)

In [15]:
#Original data visualize code
motion1 = np.load("data/interhuman_processed/motions_processed_interrope/person1/1.npz")
motion2 = np.load("data/interhuman_processed/motions_processed_interrope/person2/1.npz")
#motion1.files
motion1 = np.concatenate([motion1['joint_transl_global'], motion1['joint_rot6d_global']], axis=-1)
motion2 = np.concatenate([motion2['joint_transl_global'], motion2['joint_rot6d_global']], axis=-1)
motion = np.stack([motion1, motion2], axis = 1) # F, N, J, C
print(motion.shape)

motion = motion[0::2]
sequences = [[], []]
result_path = f"results/{1}_interrope.mp4"
name = "test"
for j in range(2):
    motion_output = motion[:,j]
    joints3d = motion_output[..., :3][:, :, [0,2,1]]
    sequences[j].append(joints3d)

for j in range(2):
    sequences[j] = np.concatenate(sequences[j], axis=0)


plot_t2m(mp_data=[sequences[0], sequences[1]], result_path=result_path, caption=name)

(477, 2, 22, 9)
239


In [2]:
motion1 = np.load("data/interhuman_processed/motions_processed_interrope/person1/1.npz")
motion2 = np.load("data/interhuman_processed/motions_processed_interrope/person2/1.npz")
#motion1.files
motion1 = np.concatenate([motion1['joint_transl_global'], motion1['joint_rot6d_global']], axis=-1)
motion2 = np.concatenate([motion2['joint_transl_global'], motion2['joint_rot6d_global']], axis=-1)
motion = np.stack([motion1, motion2], axis = 1) # F, N, J, C
motion = np.expand_dims(motion, axis=0) # 1, F, N, J, C


motion_tensor = torch.tensor(motion, dtype=torch.float32, device="cuda")  

motion_tensor = convert_intergen_notation_v2(motion_tensor) # 1, F, N, 258
print("motion_tensor.shape: ", motion_tensor.shape)
motion_numpy = motion_tensor.cpu().numpy()

#motion_numpy.shape
motion = motion_numpy.squeeze(0) # 1, F, N, 258 -> F, N, 258
#motion.shape
motion = motion[1::2]
sequences = [[], []]
result_path = f"results/{1}_interrope.mp4"
name = "test"
for j in range(2):
    motion_output = motion[:,j]
    joints3d = motion_output[:,:22*3].reshape(-1,22,3)
    #joints3d = filters.gaussian_filter1d(joints3d, 1, axis=0, mode='nearest')
    sequences[j].append(joints3d)


sequences[0] = np.concatenate(sequences[0], axis=0)
sequences[1] = np.concatenate(sequences[1], axis=0)

    
    
plot_t2m(mp_data=[sequences[0], sequences[1]],result_path=result_path, caption= name)

data.shape:  (476, 66)
data.shape:  (476, 132)
rot_data.shape:  (477, 132)
data.shape:  (476, 264)
data.shape:  (476, 268)
data.shape:  (476, 66)
data.shape:  (476, 132)
rot_data.shape:  (477, 132)
data.shape:  (476, 264)
data.shape:  (476, 268)
output.shape:  (1, 476, 2, 132)
result.shape:  torch.Size([1, 476, 2, 258])
motion_tensor.shape:  torch.Size([1, 476, 2, 258])
238


In [16]:
#InterGen processed data
motion1 = np.load("data/interhuman_processed/motions_processed/person1/1.npy")
motion2 = np.load("data/interhuman_processed/motions_processed/person2/1.npy")
motion1, root_quat_init1, root_pos_init1 = process_motion_np(motion1, 0.001, 0, n_joints=22)
motion2, root_quat_init2, root_pos_init2 = process_motion_np(motion2, 0.001, 0, n_joints=22)
r_relative = qmul_np(root_quat_init2, qinv_np(root_quat_init1))
angle = np.arctan2(r_relative[:, 2:3], r_relative[:, 0:1])

xz = qrot_np(root_quat_init1, root_pos_init2 - root_pos_init1)[:, [0, 2]]
relative = np.concatenate([angle, xz], axis=-1)[0]
motion2 = rigid_transform(relative, motion2)



result_path = f"results/{1}.mp4"
name = "test"

motion_output_both = np.stack([motion1, motion2], axis=1)
sequences = [[], []]
for j in range(2):
    motion_output = motion_output_both[:,j]

    joints3d = motion_output[:,:22*3].reshape(-1,22,3)
    #joints3d = filters.gaussian_filter1d(joints3d, 1, axis=0, mode='nearest')
    sequences[j].append(joints3d)


sequences[0] = np.concatenate(sequences[0], axis=0)
sequences[1] = np.concatenate(sequences[1], axis=0)

    
    
plot_t2m(mp_data=[sequences[0], sequences[1]],result_path=result_path, caption= name)

238
